In [49]:
import numpy as np
import pandas as pd
from cvxopt import matrix, solvers
import pickle

In [73]:
def create_gaussian_kernel(gamma=0.001):
    def kernel(x, z):
        xx = (x**2).sum(axis=1).reshape(-1,1)@np.ones((1,z.shape[0]))
        zz = (z**2).sum(axis=1).reshape(-1,1)@np.ones((1,x.shape[0]))
        return np.exp(-gamma*(xx + zz.T - 2*x@z.T))
    
    return kernel

def linear_kernel(x, z):
    return x@z.T
      
class SoftKernelSVM_QP:
    
    def __init__(self, kernel, C=1):
        self.C = C
        self.kernel = kernel
        
    def learn(self, X, y, thresh=1e-6):
        alphas = np.array(self.sol['x'])
        self.sv_idxs = (alphas > thresh).flatten()
        print(f"Got {np.count_nonzero(sv_idxs)} support vectors")
        self.sv = X[self.sv_idxs]
        self.sv_y = y[self.sv_idxs]
        self.alphas = alphas[self.sv_idxs]
        
        if (self.kernel == linear_kernel):
            self.w = self.sv.T @ (self.alphas * self.sv_y)
        
        self.b = self.sv_y - np.sum(self.kernel(self.sv,self.sv) * self.alphas * self.sv_y, axis=0)
        self.b = np.mean(self.b)
    
   # X, y need to be numpy arrays
    def fit(self, X, y):
        m, n = X.shape
        p_np = (y @ y.T) * self.kernel(X,X)
        P = matrix(p_np, tc='d') # half not required
        q = matrix(-np.ones((m, 1)), tc='d')
        A = matrix(y.T, tc='d')
        b = matrix(np.zeros(1), tc='d')
        G = matrix(np.vstack([-np.eye(m), np.eye(m)]), tc='d')
        h = matrix(np.hstack([np.zeros(m), self.C*np.ones(m)]).reshape(-1,1), tc='d')
        
        self.sol = solvers.qp(P, q, G, h, A, b)

    def predict(self, X):
        prod = np.sum(self.kernel(self.sv,X) * self.alphas * self.sv_y, axis=0) + self.b
        return np.sign(prod).astype(np.double)


In [74]:
def load_data(dir, type, digit):
    data_file = open(f'{dir}/{type}.pickle', 'rb')
    data = pickle.load(data_file)
    data_file.close()

    s = data['data'].shape
    data['data'] = data['data'].reshape(s[0], np.product(s[1:]))

    data_idx = ((data['labels'] == (digit%5)) | (data['labels'] == ((digit+1)%5))).flatten()
    data['data'] = data['data'][data_idx,:]
    data['labels'] = data['labels'][data_idx]
    data['labels'] = np.where(data['labels'] == (digit%5), 1, -1)
    
    data['data'] = data['data'].astype(np.double)
    data['labels'] = data['labels'].astype(np.double)
    
    data['data'] = np.interp(data['data'], (0,255), (0,1))
    
    return data

In [52]:
def normalize(dataset):
    new_dataset = {
        'labels': dataset['labels']
    }
    return new_dataset

## Testing

In [75]:
dpath = '../data/part2_data'
train_data = load_data(dpath, 'train_data', 9)
test_data = load_data(dpath, 'test_data', 9)


In [76]:
model = SoftKernelSVM_QP(linear_kernel)
model.fit(train_data['data'], train_data['labels'])

     pcost       dcost       gap    pres   dres
 0: -1.1712e+03 -1.0053e+04  5e+04  3e+00  5e-10
 1: -8.2018e+02 -7.0364e+03  1e+04  6e-01  5e-10
 2: -6.5638e+02 -3.7165e+03  5e+03  2e-01  5e-10
 3: -5.8967e+02 -1.3383e+03  9e+02  2e-02  4e-10
 4: -6.5370e+02 -9.3520e+02  3e+02  6e-03  4e-10
 5: -6.9062e+02 -8.0961e+02  1e+02  2e-03  4e-10
 6: -7.1076e+02 -7.5125e+02  4e+01  3e-04  5e-10
 7: -7.1909e+02 -7.3061e+02  1e+01  7e-05  5e-10
 8: -7.2208e+02 -7.2387e+02  2e+00  7e-06  5e-10
 9: -7.2264e+02 -7.2274e+02  1e-01  3e-07  5e-10
10: -7.2267e+02 -7.2268e+02  6e-03  2e-08  5e-10
11: -7.2268e+02 -7.2268e+02  1e-04  3e-10  5e-10
Optimal solution found.


In [77]:
model.learn(train_data['data'], train_data['labels'])

Got 1528 support vectors


In [56]:
model.alphas

array([[0.15641384],
       [0.99999999],
       [0.99999999],
       ...,
       [0.07483896],
       [0.77099704],
       [0.50749853]])

In [59]:
preds = model.predict(test_data['data'])

In [60]:
count = np.count_nonzero(preds == test_data['labels'].flatten())
acc = count/test_data['labels'].size
print(f"Accuracy: {count}/{test_data['labels'].size} = {acc}")

Accuracy: 1580/2000 = 0.79


Linear:
 * Accuracy: 1580/2000
 * SV's: 1528
 * 

## Scikit-Learn

In [65]:
from sklearn import svm
clf = svm.SVC()
clf.fit(train_data['data'], train_data['labels'].flatten())
preds = clf.predict(test_data['data'])

In [12]:
count = np.count_nonzero(preds == test_data['labels'].flatten())
acc = count/test_data['labels'].size
print(f"Accuracy: {count}/{test_data['labels'].size} = {acc}")

Accuracy: 1739/2000 = 0.8695


In [13]:
clf_lin = svm.SVC(kernel='linear')
clf_lin.fit(train_data['data'], train_data['labels'].flatten())
preds_lin = clf_lin.predict(test_data['data'])

count = np.count_nonzero(preds_lin == test_data['labels'].flatten())
acc = count/test_data['labels'].size
print(f"Accuracy: {count}/{test_data['labels'].size} = {acc}")

Accuracy: 1468/2000 = 0.734


In [66]:
clf.n_support_

array([950, 861], dtype=int32)

In [70]:
s1 = set(clf.support_)
s2 = set(model.sv_idxs)

1811

In [71]:
clf_lin.coef_

array([[-0.00488699, -0.00115852, -0.01225312, ..., -0.0020295 ,
        -0.00054079, -0.00621342]])

In [78]:
np.linalg.norm(clf_lin.coef_ - model.w)

1111.462445371782

In [79]:
clf_lin.coef0

0.0

In [80]:
model.b

1.5776317496953958